In [1]:
from __future__ import division
import os
import time
import math
from glob import glob
import tensorflow as tf
import numpy as np
from scipy.misc import *
from ops import *
from datasets import *


import imageio

import pdb

from msssim import tf_ms_ssim

/home/ajay/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ajay/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ajay/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ajay/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWa

In [2]:
from scipy.misc import *
print(tf.__version__)

1.9.0


In [3]:
class Finn(object):
    def __init__(self, sess, df_dim, batch_size, dropout_prob, l1_weight, ssim_weight, clipping_weight, discriminator_weight, writer_path, video_path):
        self.df_dim = df_dim
        self.batch_size = batch_size
        self.dropout_prob = dropout_prob
        self.l1_weight = l1_weight
        self.ssim_weight = ssim_weight
        self.clipping_weight = clipping_weight
        self.discriminator_weight = discriminator_weight

        self.sess = sess
        self.writer_path = writer_path
        self.filename = 'fbp'
        self.video_path = video_path


        data = generateDataSet(self.video_path)
        self.train_doublets = data["train_doublets"]
        self.train_triplets = data["train_triplets"]
        self.train_singlets = self.train_triplets[:,:,:,3:6]
        self.val_doublets = data["val_doublets"]
        self.val_targets = data["val_targets"]
        self.mean_img = data["mean_img"]

        self.input_height = self.train_doublets.shape[1]
        self.input_width = self.train_doublets.shape[2]
        self.dataset_name = 'fbp'

        self.gen_layer_depths = [32, 64, 64, 128]
        self.gen_filter_sizes = [3, 3, 3, 3]

        self.max_outputs = 100

    def discriminator(self, triplet, phase, reuse = False):
        with tf.variable_scope("discriminator") as scope:
            if reuse:
                scope.reuse_variables()

            h0 = lrelu(conv2d(triplet, self.df_dim, hh=4, ww=4, stride_w=2, stride_h=2, padding='VALID', name="d_h0_conv"))
            h1 = lrelu(bn(conv2d(h0, self.df_dim*2, hh=4, ww=4, stride_w=2, stride_h=2, padding='VALID', name="d_h1_conv"), phase, name="d_h1_bn"))
            h2 = lrelu(bn(conv2d(h1, self.df_dim*4, hh=4, ww=4, stride_w=2, stride_h=2, padding='VALID', name="d_h2_conv"), phase, name="d_h2_bn"))
            h3 = lrelu(bn(conv2d(h2, self.df_dim*8, hh=4, ww=4, stride_w=2, stride_h=2, padding='VALID', name="d_h3_conv"), phase, name="d_h3_bn"))
            h4 = conv2d(h3, 1, hh=4, ww=4, padding='SAME')
            #h4 = linear(tf.reshape(h3, [self.batch_size, -1]), 1, 'd_h3_lin')

            return tf.nn.sigmoid(h4), h4

    def generator(self, doublet):
        with tf.variable_scope("generator"):
            conv_outputs = []

            current_input = doublet
            current_inputdepth = doublet.shape[3]
            for i, outputdepth in enumerate(self.gen_layer_depths):
                result = conv_block(current_input, self.is_training, self.gen_filter_sizes[i], outputdepth, name=('g_conv_block'+str(i)) )
                conv_outputs.append(result)
                current_input = result
                current_inputdepth = outputdepth

            z = current_input

            rev_layer_depths = list(reversed(self.gen_layer_depths))
            rev_filter_sizes = list(reversed(self.gen_filter_sizes))
            rev_conv_outputs = list(reversed(conv_outputs))
#             for i in rev_conv_outputs:
#                 print(i.get_shape())

            # deconv portion
            for i, outputdepth in enumerate(rev_layer_depths[1:]): # reverse process exactly until last step

                result = bilinear_resize_deconv_block(current_input, self.is_training, rev_filter_sizes[i], outputdepth, name=('g_deconv_block'+str(i)) )
                if i <= 4:
                    result = tf.nn.dropout(result, self.dropout_prob)
                print( "result"+str(i), result.get_shape() )
                print("conc"+str(i),rev_conv_outputs[i+1].get_shape())
                stack = tf.concat([result, rev_conv_outputs[i+1]], 3)
                # print( i, stack.get_shape() )
                current_input = stack

            outputdepth = 3 # final image is 3 channel
            h = bilinear_resize_tanh_deconv_block(current_input, self.is_training, rev_filter_sizes[-1], outputdepth, name=('g_tanh_deconv') )
            return conv2d(h, outputdepth, hh=1, ww=1, mean=0.11, stddev=0.04, name='final_conv')

    def build_model(self):
        singlet_dims = [self.input_height, self.input_width, 3]
        image_dims = [self.input_height, self.input_width, 6]
        triplet_dims = [self.input_height, self.input_width, 9]

        # Set up placeholders
        self.singlets = tf.placeholder(tf.float32, [self.batch_size] + singlet_dims, name = 'singlets')
        self.doublets = tf.placeholder(tf.float32, [self.batch_size] + image_dims, name = 'doublets')
        self.triplets = tf.placeholder(tf.float32, [self.batch_size] + triplet_dims, name = 'triplets')
        self.is_training = tf.placeholder(tf.bool, (), name = 'is_training')

        sing_mean_added = self.singlets + self.mean_img

        # Sample generated frame from generator
        self.G = self.generator(self.doublets)
        g_mean_added = self.G + self.mean_img
#         g_mean_added_clipped = tf.clip_by_value(g_mean_added, 0, 1)

        # Assemble fake triplets using generated frame
        self.before, self.after = tf.split(self.doublets, [3, 3], 3)
        self.fake_triplets = tf.concat([self.before, self.G, self.after], 3)

        # Evaluate discrimator on real triplets
        self.D_real, self.D_real_logits = self.discriminator(self.triplets, self.is_training, reuse=False)
        # Use same discriminator on fake triplets
        self.D_fake, self.D_fake_logits = self.discriminator(self.fake_triplets, self.is_training, reuse=True)

        # Calculate GAN losses
        self.d_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_real_logits, labels=tf.ones_like(self.D_real)))
        self.d_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_logits, labels=tf.zeros_like(self.D_fake)))
        self.g_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_logits, labels=tf.ones_like(self.D_fake)))

        # Calculate auxiliary losses
        self.ms_ssim_loss = tf.reduce_mean(-tf.log(tf_ms_ssim(g_mean_added, sing_mean_added)))

        eps = 1e-5
        self.l1_loss = tf.reduce_mean(tf.sqrt(tf.square(self.G - self.singlets) + eps))

#         self.clipping_loss = tf.reduce_mean(tf.square(g_mean_added - g_mean_added_clipped))

        # Combine losses into single functions for the discriminator and the generator
        self.d_loss_total = self.d_loss_fake + self.d_loss_real
#         self.g_loss_total = self.discriminator_weight*self.g_loss + \
#                                 self.l1_weight*self.l1_loss + \
#                                 self.ssim_weight*self.ms_ssim_loss + \
#                                 self.clipping_weight*self.clipping_loss
        self.g_loss_total = self.discriminator_weight*self.g_loss + \
                                self.l1_weight*self.l1_loss + \
                                self.ssim_weight*self.ms_ssim_loss


        # Record relevant variables to TensorBoard
        #  - discriminator logistic output
        self.d_real_sum = tf.summary.histogram("d_real", self.D_real)
        self.d_fake_sum = tf.summary.histogram("d_fake", self.D_fake)
        #  - discriminator losses on real and fake images
        self.d_loss_real_sum = tf.summary.scalar("real_loss", self.d_loss_real)
        self.d_loss_fake_sum = tf.summary.scalar("fake_loss", self.d_loss_fake)
        self.d_loss_total_sum = tf.summary.scalar("D_loss_total", self.d_loss_total)
        #  - generator losses
        self.g_loss_sum = tf.summary.scalar("G_loss", self.g_loss)
        self.l1_loss_sum = tf.summary.scalar("l1_loss", self.l1_loss)
        self.ms_ssim_loss_sum = tf.summary.scalar("ms_ssim_loss", self.ms_ssim_loss)
#         self.clipping_loss_sum = tf.summary.scalar("clipping_loss", self.clipping_loss)
        self.g_loss_total_sum = tf.summary.scalar("G_total_loss", self.g_loss_total)
        # - sample images
        self.num_images = self.batch_size

        # clipped_G_img = tf.nn.clip_by_value(self.G + self.mean_img, 0,1)
        clipped_G_img = self.G + self.mean_img
        self.G_image = tf.summary.image("G", clipped_G_img,
            max_outputs=self.max_outputs)
        self.before_image = tf.summary.image("Z1", self.before + self.mean_img, max_outputs=self.max_outputs)
        self.after_image = tf.summary.image("Z2", self.after + self.mean_img, max_outputs=self.max_outputs)

        # Collect trainable variables for the generator and discriminator
        t_vars = tf.trainable_variables()
        self.d_vars = [var for var in t_vars if 'd_' in var.name]
        self.g_vars = [var for var in t_vars if 'g_' in var.name]

        self.saver = tf.train.Saver()

    def train(self, config):


        global_step = tf.placeholder(tf.float32, shape=[])
        g_learning_rate = tf.train.exponential_decay(config.g_learning_rate, global_step,
                                                   1, 0.96, staircase=True)
        d_learning_rate = tf.train.exponential_decay(config.d_learning_rate, global_step,
                                                   1, 0.96, staircase=True)
        g_optim_l1 = tf.train.AdamOptimizer(g_learning_rate, beta1=config.beta1
                                         ).minimize(self.l1_loss, var_list=self.g_vars)

        g_optim = tf.train.AdamOptimizer(g_learning_rate, beta1=config.beta1
                                         ).minimize(self.g_loss_total, var_list=self.g_vars)
        d_optim = tf.train.AdamOptimizer(d_learning_rate, beta1=config.beta1
                                                    ).minimize(self.d_loss_total, var_list=self.d_vars)

        tf.global_variables_initializer().run()

        self.g_sum = tf.summary.merge([self.g_loss_sum, self.l1_loss_sum, self.ms_ssim_loss_sum,
                                        self.g_loss_total_sum, self.d_loss_fake_sum, self.d_fake_sum])
        self.g_sum_l1 = tf.summary.merge([self.l1_loss_sum])
        self.d_sum = tf.summary.merge([self.d_loss_real_sum, self.d_real_sum, self.d_loss_total_sum])
        self.img_sum = tf.summary.merge([self.G_image, self.before_image, self.after_image])
        self.writer = tf.summary.FileWriter(self.writer_path + "/" + self.filename, self.sess.graph)


        train_doublets = self.train_doublets
        train_triplets = self.train_triplets
        val_doublets = self.val_doublets
        val_targets = self.val_targets
        train_singlets = self.train_singlets

        train_triplets_idx = np.arange(train_triplets.shape[0])
        np.random.shuffle(train_triplets_idx)
        # train_doublets_idx = np.arange(train_doublets.shape[0])
        # np.random.shuffle(train_doublets_idx)
        train_doublets_idx = train_triplets_idx


        counter = 1
        start_time = time.time()

        for epoch in range(config.epoch):
            print(epoch)
            batch_idx = len(train_doublets) // self.batch_size


            for idx in range(0, batch_idx):
                batch_images_idx = train_triplets_idx[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_images = train_triplets[batch_images_idx]

                batch_zs_idx = train_doublets_idx[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_zs = train_doublets[batch_zs_idx]

                batch_targets = train_singlets[batch_images_idx]


                if(config.train_gan):
                    # Update D network
                    _, summary_str = self.sess.run([d_optim, self.d_sum],
                                                   feed_dict={
                                                       self.triplets: batch_images,
                                                       self.doublets: batch_zs,
                                                       self.is_training: True,
                                                       global_step: epoch
                                                   })
                    self.writer.add_summary(summary_str, counter)

                    # Update G Network
                    _, summary_str = self.sess.run([g_optim, self.g_sum],
                                                   feed_dict={
                                                       self.doublets: batch_zs,
                                                       self.is_training: True,
                                                       self.singlets: batch_targets,
                                                       global_step: epoch
                                                   })
                    _, summary_str = self.sess.run([g_optim, self.g_sum],
                                                  feed_dict={
                                                      self.doublets: batch_zs,
                                                      self.is_training: True,
                                                      self.singlets: batch_targets,
                                                      global_step: epoch
                                                  })
                    self.writer.add_summary(summary_str, counter)
                else:
                    # Update G Network
                    _, summary_str = self.sess.run([g_optim_l1, self.g_sum_l1],
                                                   feed_dict={
                                                       self.doublets: batch_zs,
                                                       self.is_training: True,
                                                       self.singlets: batch_targets,
                                                       global_step: epoch
                                                   })
                    self.writer.add_summary(summary_str, counter)

                counter += 1

                errD_fake = self.d_loss_fake.eval({ self.doublets: batch_zs, self.is_training: True})
                errD_real = self.d_loss_real.eval({ self.triplets: batch_images, self.is_training: True})
                errG = self.g_loss.eval({self.doublets: batch_zs, self.is_training: True})
                errG_l1 = self.l1_loss.eval({self.doublets: batch_zs, self.singlets: batch_targets, self.is_training: True})
                errG_ssim = self.ms_ssim_loss.eval({self.doublets: batch_zs, self.singlets: batch_targets, self.is_training: True})
                
                
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss_total %.8f, g_loss %.8f, l1_loss %.8f, ms_ssim_loss %.8f" \
                      % (epoch, idx, batch_idx, time.time() - start_time, errD_fake+errD_real, errG, errG_l1, errG_ssim))

                if idx % 5 == 0:
                    summary_str = self.sess.run(self.img_sum,
                                                   feed_dict = {
                                                       self.doublets: train_doublets[train_doublets_idx[0:config.batch_size]],
                                                       self.is_training: True,
                                                   })
                    self.writer.add_summary(summary_str, counter)


            if np.mod(epoch, 5) == 0:
                self.save(config.checkpoint_dir, counter)

                # Save images to file
                # clipped_G_img = clip_keeping_color(self.G + self.mean_img)
                clipped_G_img = self.G + self.mean_img

                G_img = [self.sess.run(clipped_G_img,
                                           feed_dict = {
                                               self.doublets: train_doublets[k*self.batch_size:(k+1)*self.batch_size] ,
                                               self.is_training: True,
                                           }) for k in range(train_doublets.shape[0] // self.batch_size)]

                G_img = np.stack(G_img, axis=0)
                print('Saving images...')
                imageio.imwrite(config.image_dir+"mean_img.png",self.mean_img)
                [ imageio.imwrite(os.path.join(config.image_dir,"G_epoch%dimg%d.png" %
                 (epoch, i)), np.squeeze(G_img[i])) for i in range(G_img.shape[0]) ]

                if(epoch == 0):
                    # Save the targets

                    # Z_imgs = train_doublets[train_doublets_idx[0:config.batch_size]]
                    Z_imgs = train_doublets
                    [ imageio.imwrite(os.path.join(config.image_dir,"Z13_epoch%dimg%d.png" %
                     (epoch, i)), (Z_imgs[i,:,:,:3] + Z_imgs[i,:,:,3:])/2 + self.mean_img) for i in range(Z_imgs.shape[0]) ]
                    
                    Z1_imgs = train_doublets
                    [ imageio.imwrite(os.path.join(config.image_dir,"Z1_epoch%dimg%d.png" %
                     (epoch, i)), (Z1_imgs[i,:,:,:3]  + self.mean_img)) for i in range(Z1_imgs.shape[0]) ]
                    
                    Z3_imgs = train_doublets
                    [ imageio.imwrite(os.path.join(config.image_dir,"Z3_epoch%dimg%d.png" %
                     (epoch, i)), (Z3_imgs[i,:,:,3:]  + self.mean_img)) for i in range(Z3_imgs.shape[0]) ]
                    
                    # S_imgs = train_singlets[train_doublets_idx[0:config.batch_size]]
                    S_imgs = train_singlets
                    [ imageio.imwrite(os.path.join(config.image_dir,"Z2_epoch%dimg%d.png" %
                     (epoch, i)), S_imgs[i] + self.mean_img) for i in range(S_imgs.shape[0]) ]


                print('Images saved!')

    def test(self, config):
        self.load(config.checkpoint_dir)

        clipped_G_img = tf.clip_by_value(self.G + self.mean_img, 0,1)

        G_img = [self.sess.run(clipped_G_img,
                   feed_dict = {
                       self.doublets: self.val_doublets[k*self.batch_size:(k+1)*self.batch_size] ,
                       self.is_training: True,
                   }) for k in range(self.val_doublets.shape[0] // self.batch_size)]

        G_img = np.stack(G_img, axis=0)

        print('Saving images...')
        [ imageio.imwrite(os.path.join(config.test_image_dir,"G_valimageno%d.png" %
                 (i)), np.squeeze(G_img[i]+self.mean_img)) for i in range(G_img.shape[0]) ]


        Z_imgs = self.val_doublets
        [ imageio.imwrite(os.path.join(config.test_image_dir,"Z13_valimgno%d.png" %
                     (i)), (Z_imgs[i,:,:,:3] + Z_imgs[i,:,:,3:])/2 + self.mean_img) for i in range(Z_imgs.shape[0]) ]

        Z_imgs = self.val_targets
        [ imageio.imwrite(os.path.join(config.test_image_dir,"Z2_valimagno%d.png" %
                     (i)), Z_imgs[i] + self.mean_img) for i in range(Z_imgs.shape[0]) ]







    @property
    def model_dir(self):
        return "{}_{}_{}_{}".format(
            self.dataset_name, self.batch_size,
            self.input_height, self.input_width)

    def save(self, checkpoint_dir, step):
        model_name = "DCGAN.model"
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                        os.path.join(checkpoint_dir, model_name),
                        global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            counter = int(next(re.finditer("(\d+)(?!.*\d)", ckpt_name)).group(0))
            print(" [*] Success to read {}".format(ckpt_name))
            return True, counter
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0


In [4]:
class Config:

    def __init__(self, epoch, g_learning_rate, d_learning_rate, beta1, batch_size):
        self.epoch = epoch
        self.g_learning_rate = g_learning_rate
        self.d_learning_rate = d_learning_rate
        self.beta1 = beta1
        self.batch_size = batch_size
        self.checkpoint_dir = '/home/ajay/work/Sparse CT/interpolation_GAN/Checkpoint/'
        self.train_gan = True
        self.image_dir = "/home/ajay/work/Sparse CT/interpolation_GAN/output/train/"
        self.test_image_dir = "/home/ajay/work/Sparse CT/interpolation_GAN/output/test/"


In [ ]:
if __name__ == '__main__':

    cfg = Config(epoch=100, g_learning_rate=.005, d_learning_rate=0.00005, beta1=0.5, batch_size=1)
    tf.enable_eager_execution()

    run_config = tf.ConfigProto()
    run_config.gpu_options.allow_growth = True

    with tf.Session(config=run_config) as sess:

        df_dim = 64.
        batch_size = cfg.batch_size
        dropout_prob = 0.5 # probability of keeping
        l1_weight = 16.
        ssim_weight = 1.
        clipping_weight = 10.
        discriminator_weight = 1.
        writer_path = './l1w_16_ssim_84_cw_10_dw_1'
        file_path = "/home/ajay/work/Sparse CT/cancerimagingarchivedata/Walnut1/data/0_255_[128]_tubev1.npy"
#         intermediate_path = "/home/ajay/work/Sparse CT/cancerimagingarchivedata/intermediates/"
        finn = Finn(sess, df_dim, batch_size, dropout_prob, l1_weight, ssim_weight, clipping_weight, discriminator_weight, writer_path, file_path)
        finn.build_model()
        finn.train(cfg)
        finn.test(cfg)


(20, 128, 128, 3)
(18, 128, 128, 6)
(18, 128, 128, 9)
input_ (1, 128, 128, 6)
input_ (1, 128, 128, 32)
input_ (1, 64, 64, 32)
input_ (1, 64, 64, 64)
input_ (1, 32, 32, 64)
input_ (1, 32, 32, 64)
input_ (1, 16, 16, 64)
input_ (1, 16, 16, 128)
input_ (1, 16, 16, 128)
input_ (1, 16, 16, 64)
result0 (1, 16, 16, 64)
conc0 (1, 16, 16, 64)
input_ (1, 32, 32, 128)
input_ (1, 32, 32, 64)
result1 (1, 32, 32, 64)
conc1 (1, 32, 32, 64)
input_ (1, 64, 64, 128)
input_ (1, 64, 64, 32)
result2 (1, 64, 64, 32)
conc2 (1, 64, 64, 32)
input_ (1, 128, 128, 64)
input_ (1, 128, 128, 3)
input_ (1, 128, 128, 3)
input_ (1, 128, 128, 9)
input_ (1, 64, 64, 64)
input_ (1, 32, 32, 128)
input_ (1, 16, 16, 256)
input_ (1, 8, 8, 512)
input_ (1, 128, 128, 9)
input_ (1, 64, 64, 64)
input_ (1, 32, 32, 128)
input_ (1, 16, 16, 256)
input_ (1, 8, 8, 512)
